In [1]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, log_loss, matthews_corrcoef,
    balanced_accuracy_score
)
import pandas as pd
import numpy as np

In [2]:
# Set display option to show full text in all columns
pd.set_option('display.max_colwidth', None)

In [3]:
dataset = load_dataset("dair-ai/emotion", "split")

In [4]:
df_train = pd.DataFrame(dataset["train"])
df_train["label_name"] = df_train["label"].map(lambda x: dataset["train"].features["label"].names[x])

df_validation = pd.DataFrame(dataset["validation"])
df_validation["label_name"] = df_train["label"].map(lambda x: dataset["validation"].features["label"].names[x])

df_test = pd.DataFrame(dataset["test"])
df_test["label_name"] = df_train["label"].map(lambda x: dataset["test"].features["label"].names[x])

df = pd.concat([df_train, df_validation, df_test], ignore_index=True)

# Drop current encoding
df = df.drop(columns=['label'])

# Get the unique classes and create a mapping to integers
label_mapping = {label: idx for idx, label in enumerate(df['label_name'].unique())}

# Apply the mapping to create the new 'label' column
df['label'] = df['label_name'].map(label_mapping)

display(df)

,text,label_name,label
0,i didnt feel humiliated,sadness,0
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness,0
2,im grabbing a minute to post i feel greedy wrong,anger,1
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love,2
4,i am feeling grouchy,anger,1
...,...,...,...
19995,i just keep feeling like someone is being unkind to me and doing me wrong and then all i can think of doing is to get back at them and the people they are close to,sadness,0
19996,im feeling a little cranky negative after this doctors appointment,joy,5
19997,i feel that i am useful to my people and that gives me a great feeling of achievement,surprise,3
19998,im feeling more comfortable with derby i feel as though i can start to step out my shell,surprise,3


In [5]:
ordered_labels = list(df["label"].unique())
print(ordered_labels)

[np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]


In [6]:
ordered_labels_names = list(df["label_name"].unique())
print(ordered_labels_names)

['sadness', 'anger', 'love', 'surprise', 'fear', 'joy']


In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast and accurate
X = model.encode(df['text'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

In [8]:
y = df['label']

In [9]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [10]:
# Train model
clf = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=42)

In [11]:
# Predict continuous output
y_pred = clf.predict(X_test)

In [12]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

# Use label names for index and columns
cm_df = pd.DataFrame(cm, columns=ordered_labels_names, index=ordered_labels_names)

# Create a dictionary of metrics and descriptions
metrics_dict = {
    "Metric": [
        "Accuracy",
        "Precision",
        "Recall (Sensitivity)",
        "F1 Score",
    ],
    "Value": [
        accuracy,
        precision,
        recall,
        f1,
    ],
    "Description": [
        "Ratio of correct predictions to total predictions",
        "True Positives / (True Positives + False Positives)",
        "True Positives / (True Positives + False Negatives)",
        "Harmonic mean of precision and recall",
    ],
}

# Convert to DataFrame
metrics_df = pd.DataFrame(metrics_dict)

# Display nicely rounded results
metrics_df["Value"] = metrics_df["Value"].apply(lambda x: round(x, 3))

Evaluation Metrics

In [13]:
display(metrics_df)

,Metric,Value,Description
0,Accuracy,0.563,Ratio of correct predictions to total predictions
1,Precision,0.551,True Positives / (True Positives + False Positives)
2,Recall (Sensitivity),0.563,True Positives / (True Positives + False Negatives)
3,F1 Score,0.554,Harmonic mean of precision and recall


Confusion Matrix

In [14]:
display(cm_df)

,sadness,anger,love,surprise,fear,joy
sadness,752,106,30,17,67,200
anger,136,247,15,6,40,108
love,60,17,99,9,24,140
surprise,34,12,4,31,24,52
fear,82,46,11,15,206,85
joy,190,76,64,14,64,917
